[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/03_fastai_segmentation.ipynb)

# Vorbereitung

Laufzeit-Typ ändern. Python-Pakete von fast.ai installieren und importieren.

In [ ]:
!pip install --upgrade fastai fastcore

In [ ]:
from fastai.vision.all import *

**Wichtig!** Wenn die folgende Zelle "False" ausgibt, ist keine GPU vorhanden -> Bitte prüfe die o. g. Schritte noch einmal und starte ggf. das Notebook neu. Bei "True" hast du eine GPU-Instanz und kannst weiter machen.

In [ ]:
torch.cuda.is_available()

# Daten laden

Als Grundlage dient der Camvid Datensatz aus dem Paper *Semantic Object Classes in Video: A High-Definition Ground Truth Database*, das ebenfalls in Fast.ai integriert ist.

In [ ]:
path = untar_data(URLs.CAMVID_TINY)

In [ ]:
path.ls()

Die Bilder sind im `images` Ordner und die dazugehörigen Segmentation-Masken entsprechen den `labels`.

Die Masken besitzen für jeden Pixel einen Integer Wert. Diese Werte sind in der `codes.txt` abgespeichert.

In [ ]:
codes = np.loadtxt(path/'codes.txt', dtype=str)

In [ ]:
codes

Ähnlich wie beim Cats-vs-Dogs Classifier sind die Images für das Labeling der Daten nach einer bestimmten Syntax kodiert. In diesem Fall haben die Segmentation-Masken den selben Namen wie die Bilder und sind um ein _P ergänzt. Deshalb definieren wir folgende Labelfunktion. 

In [ ]:
(path/"labels").ls()[0]

In [ ]:
def label_func(fn): return path/"labels"/f"{fn.stem}_P{fn.suffix}"

# Model trainieren

In [ ]:
camvid = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                   get_items = get_image_files,
                   get_y = label_func,
                   splitter=RandomSplitter(),
                   batch_tfms=aug_transforms(size=(120,160)))
dls = camvid.dataloaders(path/"images", path=path, bs=8)

In [ ]:
dls.show_batch(max_n=6)

# Model trainieren

Für Segmentation nutzt man ein spezielles Modell namens UNet, statt des klassischen CNNs. Deshalb nutzen wir für dieses Beispiel den `unet_learner`

In [ ]:
learn = unet_learner(dls, resnet34, metric="error_rate")
learn.fine_tune(8)

Zur Visualisierung der Ergebnisse werden die Pixel entsprechend der Prediction des Modells eingefärbt. In jedem Bilderpaar entspricht das linke Bild dem Ziel und die rechte Seite der Prediction des Modells.

In [ ]:
learn.show_results(max_n=6, figsize=(7,8))